In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Setting up Spark 
!pip install pyspark


In [ ]:
# Entery Point of Spark 
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Exploratory Analysis") \
    .getOrCreate()

In [ ]:
# importing DataSet
P_Data_Set = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')
P_Data_Set

In [ ]:
# Exploring Data_Set, Display summary statistics
P_Data_set.head(5)
P_Data_Set.tail(2)
P_Data_set.describe().show()

In [ ]:
# datatype of columns
P_Data_Set.printSchema()

#PreProcessing of Data such as Droping Duplicates, removing null values 

In [ ]:
P_Data_Set=P_Data_Set.dropDuplicates()
P_Data_Set = P_Data_Set.dropna()
P_Data_Set.count()

In [ ]:
# converting the Data set into a data frame
P_Data_Set = P_Data_Set.toDF(*(c.replace(' ', '_') for c in parking.columns))
parking.show(3)

In [ ]:
# creating a local temporary view of the table, to avoid changed in the orginal data set 
parking.createOrReplaceTempView("Ptable")
spark.sql('Select * from Ptable') 

In [ ]:
# Total number of tickets for each year
sql_ticket_year = spark.sql("select year(Issue_Date) as year, count(Summons_Number) as no_of_tickets from Ptable group by year order by year")
sql_ticket_year.show(50)
sql_ticket_year.count()
# error while counting the ticket number 

In [ ]:
# Specfying the scope of the analysis to 2015 data 
parking.createOrReplaceTempView("tble_view2015")
parking=spark.sql("select * from tble_view2015 where year(TO_DATE(CAST(UNIX_TIMESTAMP(Issue_Date,'MM/dd/yyyy') AS TIMESTAMP))) = 2015 ")
parking.count()
# error while filting the data  

In [ ]:
P_Data_Set.createOrReplaceTempView("tble_view2015")
#Showing distribution of tickets over the year 
Distribution_on_years= spark.sql("SELECT year(Issue_Date) as year,month(Issue_Date) as month,count(*) as Ticket_Frequency FROM tble_view2015 GROUP BY year(Issue_Date),month(Issue_Date) order by Ticket_Frequency desc")

In [ ]:
Distribution_on_years.show()
Number_of_Violations_by_month = Distribution_on_years.toPandas()

In [ ]:
Number_of_Violations_by_month.plot(x= 'month', y='Ticket_Frequency', kind='line')
plt.title("Violations on the basis of month in 2015")
plt.xlabel('month')
plt.ylabel('Ticket_Frequency')
plt.show()